# Deep Learning Demo

In [ ]:
from main import *
import os
import cv2
import json
import helpers
import warnings
from PIL import Image
from pathlib import Path
from types import SimpleNamespace
import configuration_demo as config
from matplotlib import pyplot as plt
from data_augmentation import RandomSTRAugmentation

warnings.filterwarnings("ignore")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

%reload_ext autoreload
%autoreload 2

In [ ]:
args = {"dataset": "SoccerNet", "part": "test", "train_str": False}
args = SimpleNamespace(**args)

os.makedirs(config.dataset["SoccerNet"]["working_dir"], exist_ok=True)

image_dir = os.path.join(
    config.dataset["SoccerNet"]["root_dir"],
    config.dataset["SoccerNet"][args.part]["images"],
)

soccer_ball_list = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["soccer_ball_list"],
)

features_dir = config.dataset["SoccerNet"][args.part]["feature_output_folder"]
full_legibile_path = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["legible_result"],
)

illegible_path = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["illegible_result"],
)

gt_path = os.path.join(
    config.dataset["SoccerNet"]["root_dir"],
    config.dataset["SoccerNet"][args.part]["gt"],
)

input_json = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["pose_input_json"],
)

output_json = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["pose_output_json"],
)

crops_destination_dir = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["crops_folder"],
    "imgs",
)

str_result_file = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["jersey_id_result"],
)

final_results_path = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["final_result"],
)

crop_image_dir = os.path.join(
    config.dataset["SoccerNet"]["working_dir"],
    config.dataset["SoccerNet"][args.part]["crops_folder"],
)

## Legibility Classifer

In [ ]:
print("Determine soccer ball")
success = helpers.identify_soccer_balls(image_dir, soccer_ball_list)
print("Done determine soccer ball")

print("\nGenerate features")
command = f"conda run -n {config.reid_env} --live-stream python {config.reid_script} --tracklets_folder {image_dir} --output_folder {features_dir}"
os.system(command)
print(f"Done generating features")

print("\nIdentify and remove outliers")
command = f"python gaussian_outliers.py --tracklets_folder {image_dir} --output_folder {features_dir}"
os.system(command)
print(f"Done removing outliers")

print("\nClassifying Legibility:")
try:
    legible_dict, illegible_tracklets = get_soccer_net_legibility_results(
        args, use_filtered=True, filter="gauss", exclude_balls=True
    )
except Exception as error:
    print(f"Failed to run legibility classifier:{error}")
    raise error
print("Done classifying legibility")

##### Plot Illegible Samples

In [ ]:
with open(illegible_path, "r") as file:
    data = json.load(file)

image_paths = []
directory_paths = data.get("illegible", [])
for directory in directory_paths:
    dir_full_path = os.path.join(image_dir, directory)
    for filename in os.listdir(dir_full_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            image_paths.append(os.path.join(dir_full_path, filename))

images = []
image_paths = image_paths[:5]
for path in image_paths:
    if os.path.exists(path):
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (50, 128))
        images.append(img_resized)
    else:
        print(f"Image at path {path} not found!")

if len(images) == 0:
    print("Failed to plot images: no illigble images exist")
else:
    fig, axes = plt.subplots(1, len(images), figsize=(10, 5))

    if len(images) == 1:
        axes = [axes]

    for i in range(len(images)):
        axes[i].imshow(images[i])
        axes[i].axis("off")

    fig.suptitle("Illegible Examples")
    plt.tight_layout()
    plt.show()

## Pose Detector

In [ ]:
print("Generating json for pose")
try:
    with open(full_legibile_path, "r") as openfile:
        legible_dict = json.load(openfile)
    generate_json_for_pose_estimator(args, legible=legible_dict)
except Exception as error:
    print(f"Failed to run HPE: {error}")
    raise error
print("Done generating json for pose")

print("\nDetecting pose")
command = f"conda run -n {config.pose_env} --live-stream python pose.py {config.pose_home}/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_huge_coco_256x192.py \
            {config.pose_home}/checkpoints/vitpose-h.pth --img-root / --json-file {input_json} \
            --out-json {output_json}"
os.system(command)
print("Done detecting pose")

print("\nGenerate crops")
try:
    Path(crops_destination_dir).mkdir(parents=True, exist_ok=True)
    with open(full_legibile_path, "r") as outfile:
        legible_results = json.load(outfile)
    helpers.generate_crops(output_json, crops_destination_dir, legible_results)
except Exception as e:
    print(f"Failed to run generate crops: {error}")
    raise error
print("Done generating crops")

##### Plot Crops

In [ ]:
images = []
for path in os.listdir(crops_destination_dir):
    if len(images) > 5:
        break
    if not path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
        continue
    full_path = os.path.join(crops_destination_dir, path)
    if os.path.exists(full_path):
        img = cv2.imread(full_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img_resized)
    else:
        print(f"Image at path {full_path} not found!")

if len(images) == 0:
    print("Failed to plot images: no cropped images exist")
else:
    fig, axes = plt.subplots(1, len(images), figsize=(10, 5))

    if len(images) == 1:
        axes = [axes]

    for i in range(len(images)):
        axes[i].imshow(images[i])
        axes[i].axis("off")

    fig.suptitle("Illegible Examples", y=0.9)
    plt.tight_layout()
    plt.show()

## Visualizing Data Augmentation
!!Not applied during test!!

In [ ]:
images = []
input_data = crops_destination_dir
for path in os.listdir(input_data):
    if len(images) > 5:
        break
    if not path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
        continue
    full_path = os.path.join(input_data, path)
    if os.path.exists(full_path):
        img = Image.open(full_path).convert("RGB")
        images.append(img)
    else:
        print(f"Image at path {full_path} not found!")

augmented_images = []
for img in images:
    aug = RandomSTRAugmentation(mag_range=[0, 1])
    augmented_images.append(aug(img))

fig, axes = plt.subplots(1, len(images), figsize=(10, 5))

if len(augmented_images) == 1:
    axes = [axes]

for i in range(len(augmented_images)):
    axes[i].imshow(augmented_images[i])
    axes[i].axis("off")

fig.suptitle("Augmented Examples", y=0.7)
plt.tight_layout()
plt.show()

## Scene Text Recognition

In [ ]:
print("Predict numbers")

command = f"conda run -n {config.str_env} --live-stream python str.py  {config.dataset['SoccerNet']['str_model']}\
	--data_root={crop_image_dir} --batch_size=1 --inference --result_file {str_result_file}"
os.system(command)
print("Done predict numbers")

analysis_results = None
results_dict, analysis_results = helpers.process_jersey_id_predictions(
    str_result_file, useBias=True
)

consolidated_dict = consolidated_results(
    crop_image_dir, results_dict, illegible_path, soccer_ball_list=soccer_ball_list
)

with open(final_results_path, "w") as f:
    json.dump(consolidated_dict, f)

##### Visualizing Result

In [ ]:
import random


with open(final_results_path, "r") as f:
    data = json.load(f)

result = []
for key in data.keys():
    if len(result) == 3:
        break
    try:
        key = int(key)
    except:
        continue

    dir_full_path = os.path.join(image_dir, str(key))
    image_path = os.path.join(dir_full_path, random.choice(os.listdir(dir_full_path)))
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (50, 128))
    result.append((img_resized, data[str(key)]))

fig, axes = plt.subplots(1, len(result), figsize=(6, 5))

if len(result) == 1:
    axes = [axes]

for i in range(len(result)):
    axes[i].imshow(result[i][0])
    axes[i].axis("off")
    axes[i].set_title(f"Predicted {result[i][1]}")

fig.suptitle("Predicted Resulls")
plt.tight_layout()
plt.show()